In [2]:
#General use
import numpy as np
from astropy.io import fits
from astropy.cosmology import WMAP9 as cosmo

#For creating a list of .fits files to perform photometry on
import os

#For reading and creating tables
from astropy.table import Table, unique, vstack, join
import astropy.io.ascii as asc

#For performing photometry
from astropy import units as u
from photutils import aperture_photometry, SkyCircularAperture
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS

#For generating a plot
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages


In [11]:
def Conv_error(val, err):

    '''
    Find error in the conversion factor from kpc^2 to arcmin^2 using a black box error method.

    Args:
        val (float): A redshift value
        err (float): Error in the redshift
    
    Returns:
        error (float): Error in the conversion factor
    '''

    diff = cosmo.kpc_comoving_per_arcmin(val + err)**2 - cosmo.kpc_comoving_per_arcmin(val - err)**2
    error = abs(.5 * diff.value)
    return(error)

def lum_dist_error(val, err):
    
    '''
    Find error in the luminosity distance using a black box error method
    
    Args:
        val (float): A redshift value
        err (float): Error in the redshift
    
    Returns:
        error (float): Error in the luminosity distance
    '''
    
    diff = cosmo.luminosity_distance(val + err).cgs - cosmo.luminosity_distance(val - err).cgs
    error = abs(.5 * diff.value)
    return(error)

def Flux_error(pho_val, pho_err, conv_val, conv_err):
    
    '''
    Find error in the calculated flux due to error in the measured photon counts per second
    and flux conversion factor.
    
    Args:
        pho_val  (float): Average photon counts per second
        pho_err  (float): Error in pho_val
        conv_val (float): A conversion factor from average photon counts to flux
        conv_err (float): Error in conv_val
    
    Returns:
        error (float): Error in the associated flux value
    '''
    
    error = np.sqrt((pho_val * conv_err)**2 + (conv_val * pho_err)**2)
    return(error)

def Luminosity_error(flux_val, flux_err, dist_val, dist_err):
    
    '''
    Find error in the calculated Luminosity due error in the flux and luminosity distance
    
    Args:
        flux_val (float): Average photon counts per second
        flux_err (float): Error in val
        dist_val (float): Comoving distance
        dist_err (float): Error in comoving distance
    
    Returns:
        error (float): Error in the luminosity
    '''
    
    error = np.sqrt((4 * np.pi * (dist_val**2) * flux_err)**2
                    + (8 * np.pi * dist_val * flux_val * dist_err)**2)
    
    return(error)

def Sbrightness_error(lum_val, lum_err, conv_val, conv_err):
    
    '''
    Find the error in surface brightness
    
    Args:
        lum_val  (float): Luminosity of a supernova environment
        lum_err  (float): Error in the luminosity
        conv_val (float): Conversion factor from arcmin^2 to Kpc^2
        conv_err (float): Error in the conversion factor
    
    Returns:
        error (float): Error in the Surface brightness
    '''
    
    error = np.sqrt((conv_val * lum_err / 3600)**2
                    + (lum_val * conv_err / 3600)**2)
    
    return(error)

def zero_check(fits_file, cordinate, r):
    
    '''
    Perform photometry on a wt type .fits file and checks if there are data pixels in an
    aperture.
    
    Args:
        fits_file (str)      : The file path of a .fits file
        cordinate (SkyCoord) : The (ra, dec) of a supernova in degrees
        r         (Quantity) : The radius of a photometry aperture in arcmin.
    
    Returns:
        in_file         (bool): [supernova name(string), photometry value (float), exposure time (float)]
        'no check file' (str) : Returned if there is no check file
    '''
    
    if os.path.isfile(fits_file.replace('d-int', 'd-exp')):
        hdulist = fits.open(fits_file.replace('d-int', 'd-exp'))
        aperture = SkyCircularAperture(cordinate, r)
        phot_table = aperture_photometry(hdulist[0], aperture)
        in_file = phot_table[0][0] != 0
        return(in_file)
    
    else:
        return('no check file')

def photometry(fits_file, radius, cord):
    
    '''
    Perform photometry on an int type .fits file.
    
    Args:
        fits_file (str)  : The file path of an int type .fits file
        radius    (float): Radius of the desired aperture in kpc
        cord      (dict) : A dictionary of sn coordinates {sn (str): coordinates (skycord)}
    
    Returns:
        results (list): [supernova name (str), photometry value (float),
                         exposure time (float)]
        
        results (list): ['error' (str), fits file path (str),
                         error description (str)]
    '''
    
    results = []
    
    with fits.open(fits_file) as int_file:
        if os.path.isfile(fits_file.replace('d-int', 'd-skybg')):
            with fits.open(fits_file.replace('d-int', 'd-skybg')) as skybg_file:

                wcs = WCS(fits_file)
                for sn in cord:
                
                    #Define the SN location in pixels
                    w = wcs.all_world2pix(cord[sn].ra, cord[sn].dec, 1)
                    
                    #Make sure the sn is located in the image
                    if 0 < w[0] < 3600 and 0 < w[1] < 3600:
                        
                        #get exposure time and create an array of the error in each pixel
                        exp_time = int_file[0].header['EXPTIME']
                        error = np.sqrt(int_file[0].data * exp_time) / exp_time
                        
                        #Find arcmin of a 1kpc radius region
                        r = radius * u.kpc / cosmo.kpc_comoving_per_arcmin(float(red[sn]))
                        
                        #Create an aperture
                        aperture = SkyCircularAperture(cord[sn], r)
                        
                        #Subtract the skybg file
                        sky_subtracted = int_file
                        sky_subtracted[0].data = int_file[0].data - skybg_file[0].data
                        
                        #Perform photometry
                        phot_table = aperture_photometry(sky_subtracted[0], aperture, error = error)
                        
                        if phot_table[0][0] != 0:
                            results.append([sn, exp_time, phot_table])
                            
                        else:
                            check = zero_check(fits_file, cord[sn], r)
                            if check == True:
                                results.append([sn, exp_time, phot_table])
                            
                            else:
                                results.append(['error', fits_file, check])
                         
            if results == []:
                results.append(['error', fits_file, 'no supernova found'])
                return(results)
            
            else:
                return(results)
        
        else:
            results.append(['error', fits_file, 'no skybg file'])
            return(results)

def create_tables(uv_type, directory, radius, cord):
    
    '''
    Perform photometry on a directory of .fits files and create two tables.
    The first table contains the redshift, exposure time, luminosity, and surface 
    brightness for various supernova, along with the associated error values, in 
    either the far or near UV. The second table is a log outlining any files that
    do not contain a supernova or are missing checkfiles.
    
    Args:
        uv_type   (str)  : Specifies which type of uv to create a table for.
                            Use either 'NUV' or 'FUV'.
        directory (str)  : A directory containing .fits files
        radius    (float): Radius of the desired photometry aperture in kpc
        cord      (dict) : A dictionary of sn coordinates {sn (str): coordinates (skycord)}
    
    Returns:
        results               (list): [First table (Table), Second table (Table)]
    '''
    
    label = uv_type + ' ' + str(radius) + 'kpc '
    
    #Define the tables that will be returned by the function
    log = Table(names = ['File Path', 'Issue'], dtype = [object, object])
    out = Table(names = ['sn',
                         'Redshift',
                         'Redshift Error',
                         label + 'Exposure Time',
                         'Flux',
                         'Flux Error',
                         label + 'Luminosity',
                         label + 'Luminosity Error',
                         label + 'Surface Brightness',
                         label + 'Surface Brightness Error'],
                
                dtype = ('S70', 'float64', 'float64', 'float64', 'float64',
                         'float64', 'float64', 'float64', 'float64', 'float64'))
    
    out['Redshift'].unit = u.dimensionless_unscaled
    out['Redshift Error'].unit = u.dimensionless_unscaled
    out[label + 'Exposure Time'].unit = u.s
    out['Flux'].unit =  u.erg / u.s / u.Angstrom / u.kpc / u.kpc / u.cm / u.cm / np.pi
    out['Flux Error'].unit = u.erg / u.s / u.Angstrom / u.kpc / u.kpc / u.cm / u.cm / np.pi
    out[label + 'Luminosity'].unit = u.erg / u.s / u.Angstrom / u.kpc / u.kpc
    out[label + 'Luminosity Error'].unit = u.erg / u.s / u.Angstrom / u.kpc / u.kpc
    out[label + 'Surface Brightness'].unit = u.erg / u.s / u.Angstrom / u.arcsec / u.arcsec
    out[label + 'Surface Brightness Error'].unit = u.erg / u.s / u.Angstrom / u.arcsec / u.arcsec
    
    #Set parameters that are specific to NUV or FUV observations
    if 'N' in uv_type.upper():
        file_key = "nd-int" #A string located in the file name of all the files that will be analyzed
        flux_conv = 2.06 * 1e-16 #A conversion factor from counts per second to flux
    
    elif 'F' in uv_type.upper():
        file_key = "fd-int"
        flux_conv = 1.40 * 1e-15
    
    #Create a list of files to perform photometry on
    file_list = []
    for path, subdirs, files in os.walk(directory):
        for name in files:
            if file_key in name and len(name.split('.')) < 3:
                file_list.append(os.path.join(path, name))
    
    #Perform photometry on each .fits file
    for fits_file in file_list:
        p = photometry(fits_file, radius, cord)
        
        for elt in p:
            if elt[0] == 'error':
                log.add_row([elt[1], elt[2]])
            
            else:
                #We calculate the values to be entered in the table
                redshift = float(red[elt[0]])
                peculiar_redshift = np.sqrt((1 + (300 / 299792.458)) / (1 - (300 / 299792.458))) - 1
                redshift_err = np.sqrt((redshift / 1000)**2 + (peculiar_redshift)**2)
                
                arcmin = cosmo.kpc_comoving_per_arcmin(redshift).value**2 #kpc^2 per arcmin^2
                arcmin_err = Conv_error(redshift, redshift_err)
                
                photom = elt[2][0][0] #The photometry value
                photom_err = elt[2][0][1]
                print('\n fits file:', fits_file)
                print('sn:', elt[0])
                print('Photometry value:', photom)
                print('Photometry error:', photom_err, flush = True)
                
                flux = flux_conv * photom #convert cps to flux using the conversion factor
                flux_err = Flux_error(photom, photom_err, flux_conv, flux_conv_err)
                
                ldist = cosmo.luminosity_distance(redshift).cgs.value #Luminosity Distance (cm)
                ldist_err = lum_dist_error(redshift, redshift_err)
                
                lum = flux * 4 * np.pi * (ldist**2) #luminosity = flux*4*pi*r^2
                lum_err = Luminosity_error(flux, flux_err, ldist, ldist_err)
                
                sbrightness = lum * arcmin / 3600
                sbrightness_err = Sbrightness_error(lum, lum_err, arcmin, arcmin_err)
                
                out.add_row([elt[0], redshift, redshift_err, elt[1], flux, flux_err,
                             lum, lum_err, sbrightness, sbrightness_err])
    
    out.sort(label + 'Surface Brightness Error')
    out_unique = unique(out, keys = 'sn')
    out_unique.sort('sn')
    
    return([out_unique, log])

def create_plots(data_table, uv_type):
    
    '''
    Create plots of UV surface brightness vs redshift in units of erg s-1 A-1 arcsec-2
    and erg s-1 A-1 kpc-2. Plots are created using results from the create_table() function.
    Both plots are saved to the working directory.
    
    Args:
        data_table  (Table): The first table returned by create_table()
        uv_type   (str)  : Specifies if the data is for either the 'NUV' or 'FUV'.
    Returns:
        None
    '''
    
    #We create a plot of the results and write them to a pdf file
    
    label = uv_type + ' 1kpc '
    
    if 'N' in uv_type.upper(): plot_name = 'NUV Surface Brightness of SN Local Enviornments'
    elif 'F' in uv_type.upper(): plot_name = 'FUV Surface Brightness of SN Local Enviornments'
    
    plt.figure(1)
    plt.xlabel('Redshift')
    plt.ylabel(str(data_table[label + 'Surface Brightness'].unit))
    plt.title(plot_name)
    
    plt.figure(2)
    plt.xlabel('Redshift')
    plt.ylabel(str(data_table[label + 'Luminosity'].unit))
    plt.title(plot_name)
    
    for row in data_table:
        if row[label + 'Surface Brightness'] != 0:
            sigma = row['Flux'] / row['Flux Error']
            if sigma <= 3:
                
                plt.figure(1)
                plt.semilogy(row['Redshift'],
                             row[label + 'Surface Brightness'],
                             marker = u'$\u21a7$',
                             markeredgecolor='lightgrey',
                             color = 'lightgrey')
                
                plt.figure(2)
                plt.semilogy(row['Redshift'],
                             row[label + 'Luminosity'],
                             marker = u'$\u21a7$',
                             markeredgecolor='lightgrey',
                             color = 'lightgrey')
    
    for row in data_table:
        if row[label + 'Surface Brightness'] != 0:
            sigma = row['Flux'] / row['Flux Error']
            if sigma > 3:
                error = (row[label + 'Surface Brightness Error'], row[label + 'Luminosity Error'])
                
                plt.figure(1)
                plt.semilogy(row['Redshift'],
                             row[label + 'Surface Brightness'],
                             marker = '.',
                             color = 'black')
                
                plt.errorbar(row['Redshift'],
                             row[label + 'Surface Brightness'],
                             yerr = error[0],
                             color = 'black',
                             linestyle = '')
                
                plt.figure(2)
                plt.semilogy(row['Redshift'],
                             row[label + 'Luminosity'],
                             marker = '.',
                             color = 'black')
                
                plt.errorbar(row['Redshift'],
                             row[label + 'Luminosity'],
                             yerr = error[1],
                             color = 'black',
                             linestyle = '')
    
    if not os.path.exists('./plots/'):
            os.makedirs('./plots/')
    
    plt.figure(1)
    plt.savefig('./plots/' + uv_type + ' plot (arcsec).pdf')

    plt.figure(2)
    plt.savefig('./plots/' + uv_type + ' plot (kpc).pdf')

    plt.close("all")


In [12]:
#User set parameters:
region_file = "observed_target_info.reg" #A region file with supernova coordinates and redshifts

fits_directory = 'fits files' #The directory to look for .fits files in

output_file = "output" #The output file name without an extension.

flux_conv_err = 0 #Error in flux_conv

In [13]:
#Create dictionaries for the coordinates (in degrees) and redshift of supernova
    #by using values from the .reg file and from Friedman data table.csv

cord_dict, red = {}, {}

region_table = asc.read(region_file, data_start = 2, delimiter = "#", header_start = 2)
for row in region_table:
    sn = row[1].split(",")[0].strip('text={}')
    cordinates = row[0].strip('point()').replace(',',' ')
    redshift = row[1].split(",")[2].strip('}').replace('z=','')
    
    cord_dict[sn] = SkyCoord(cordinates, unit = (u.hourangle, u.deg))
    red[sn] = redshift

friedman_table = Table.read('Friedman data table.csv', format = 'csv')
for row in friedman_table:
    red[row['SN']] = row['z']
    cord_dict[row['SN']] = SkyCoord(ra = row['RAj2000'] * u.degree, dec = row['DEj2000'] * u.degree)


In [14]:
print('We create just one table as a test case\n', flush = True)
nuv_1kpc = create_tables('NUV', fits_directory, 1, cord_dict)

We create just one table as a test case


 fits file: fits files/GALEX/2422868567306272768/MISDR1_18848_0459-nd-int.fits
sn: SN2005eq
Photometry value: 0.0128699045257
Photometry error: 0.00389576352841

 fits file: fits files/GALEX/3116282172432973824/GI3_046017_NGC5338-nd-int.fits
sn: SN2006E
Photometry value: 1.42964450771
Photometry error: 0.0393351496833

 fits file: fits files/GALEX/3832495250874564608/MISGCSN1_33665_0222-nd-int.fits
sn: SN2006E
Photometry value: 1.29151367823
Photometry error: 0.0310159532917

 fits file: fits files/GALEX/6371267678664916992/AIS_10_sg74-nd-int.fits
sn: SN2012bp
Photometry value: 0.0144848311894
Photometry error: 0.0127386458279

 fits file: fits files/GALEX/6374891607786782720/AIS_113_sg17-nd-int.fits
sn: SN2006ac
Photometry value: 0.240323621965
Photometry error: 0.0345341418585

 fits file: fits files/GALEX/6376756456816902144/AIS_166_sg89-nd-int.fits
sn: SN2007rx
Photometry value: 0.0026108545057
Photometry error: 0.00730247188331

 fits file

/Users/Daniel/anaconda/lib/python3.4/site-packages/ipykernel/__main__.py:155: RuntimeWarning: invalid value encountered in sqrt


In [15]:
nuv_1kpc[0]

sn,Redshift,Redshift Error,NUV 1kpc Exposure Time,Flux,Flux Error,NUV 1kpc Luminosity,NUV 1kpc Luminosity Error,NUV 1kpc Surface Brightness,NUV 1kpc Surface Brightness Error
,,,s,0.31831 erg / (Angstrom cm2 kpc2 s),0.31831 erg / (Angstrom cm2 kpc2 s),erg / (Angstrom kpc2 s),erg / (Angstrom kpc2 s),erg / (Angstrom arcsec2 s),erg / (Angstrom arcsec2 s)
bytes560,float64,float64,float64,float64,float64,float64,float64,float64,float64
SN2005eq,0.028977,0.00100161272456,1693.05,2.65120033229e-18,8.02527286852e-19,5.20873683133e+36,1.61906486334e+36,1.89867105584e+36,6.04415457784e+35
SN2005iq,0.034044,0.0010017721185,112.0,1.36728270358e-17,5.2975351687e-18,3.73613983189e+37,1.46502122615e+37,1.87563645693e+37,7.43592711886e+36
SN2006E,0.002686,0.00100119708253,2600.15,2.66051817715e-16,6.38928637809e-18,4.31361544791e+36,3.22418808612e+36,1.36654991055e+34,1.44214786509e+34
SN2006ac,0.023106,0.00100146006945,213.0,4.95066661248e-17,7.11403322286e-18,6.12973392056e+37,1.03353325712e+37,1.42435118451e+37,2.69754287824e+36
SN2007rx,0.0301,0.00100164584233,248.0,5.37836028175e-19,1.50430920796e-18,1.14208783462e+36,3.1953266909e+36,4.48982201182e+35,1.25651047348e+36
SN2012,0.025,0.00100150555838,248.0,5.60916781954e-17,7.00801225257e-18,8.15368521693e+37,1.21690019415e+37,2.21615685987e+37,3.74939432201e+36
SN2012bp,0.02829,0.00100159308482,528.65,7.58387088971e-18,1.92401061124e-18,1.41869973713e+37,3.74258395934e+36,4.93057535011e+36,1.34618553315e+36


In [ ]:
#Calculate values used in the ouptut tables. Individual tables are generated
    #for nuv and fuv using both a 1kpc and 5kpc aperture
print('Calculating NUV values using a 1kpc radius aperture\n', flush = True)
nuv_1kpc = create_tables('NUV', fits_directory, 1, cord_dict)

print('Calculating FUV values using a 1kpc radius aperture\n', flush = True)
fuv_1kpc = create_tables('FUV', fits_directory, 1, cord_dict)

print('Calculating NUV values using a 5kpc radius aperture\n', flush = True)
nuv_5kpc = create_tables('NUV', fits_directory, 5, cord_dict)

print('Calculating FUV values using a 5kpc radius aperture\n', flush = True)
fuv_5kpc = create_tables('FUV', fits_directory, 5, cord_dict)

#Create plots and save them to file
print('Creating Plots\n', flush = True)
create_plots(nuv_1kpc[0], 'NUV')
create_plots(fuv_1kpc[0], 'FUV')

#We mantain a copy of the orignal tables befor manipulation.
    #This allows for the plots to be recreated multiple times
    #without recreating the source tables
nuv_1 = nuv_1kpc
fuv_1 = fuv_1kpc
nuv_5 = nuv_5kpc
fuv_5 = fuv_5kpc

#Create a table with the information desired to be in the .csv
print('Joining Tables\n', flush = True)
nuv_1[0].keep_columns(['sn',
                       'NUV 1kpc Luminosity',
                       'NUV 1kpc Luminosity Error',
                       'NUV 1kpc Surface Brightness',
                       'NUV 1kpc Surface Brightness Error'])

fuv_1[0].keep_columns(['sn',
                       'FUV 1kpc Luminosity',
                       'FUV 1kpc Luminosity Error',
                       'FUV 1kpc Surface Brightness',
                       'FUV 1kpc Surface Brightness Error'])

nuv_5[0].remove_columns(['Flux',
                         'Flux Error',
                         'NUV 5kpc Luminosity',
                         'NUV 5kpc Luminosity Error'])

fuv_5[0].remove_columns(['Flux',
                         'Flux Error',
                         'FUV 5kpc Luminosity',
                         'FUV 5kpc Luminosity Error'])

merged_nuv_table = join(nuv_1[0], nuv_5[0], join_type = 'outer', keys = 'sn')
merged_fuv_table = join(fuv_1[0], fuv_5[0], join_type = 'outer', keys = 'sn')

merged_table = join(merged_nuv_table, merged_fuv_table, join_type = 'outer', keys = ['sn'])
merged_table.rename_column('NUV 5kpc Exposure Time','NUV Exposure Time')
merged_table.rename_column('FUV 5kpc Exposure Time','FUV Exposure Time')
merged_table.rename_column('Redshift_1', 'Redshift')
merged_table.rename_column('Redshift Error_1', 'Redshift Error')

for row in merged_table:
    if str(row["Redshift"]) == "--":
        row["Redshift"] = row["Redshift_2"]
        row["Redshift Error"] = row["Redshift Error_2"]

merged_table.remove_columns(["Redshift_2", "Redshift Error_2"])

#rearrange the column order
final_table = merged_table['sn', 'Redshift', 'Redshift Error',
                           'NUV Exposure Time',
                           'NUV 1kpc Luminosity',
                           'NUV 1kpc Luminosity Error',
                           'NUV 1kpc Surface Brightness',
                           'NUV 1kpc Surface Brightness Error',
                           'NUV 5kpc Surface Brightness',
                           'NUV 5kpc Surface Brightness Error',
                           'FUV Exposure Time',
                           'FUV 1kpc Luminosity',
                           'FUV 1kpc Luminosity Error',
                           'FUV 1kpc Surface Brightness',
                           'FUV 1kpc Surface Brightness Error',
                           'FUV 5kpc Surface Brightness',
                           'FUV 5kpc Surface Brightness Error',]

final_table.sort('sn')

merged_log = vstack([nuv_5[1],fuv_5[1]])
merged_log.sort('File Path')

#Write data to an output file
    #The csv format does not have an 'overwrite' kwarg,
    #but an existing file will be overwritten

final_table.write(output_file + '.fits', overwrite = True, format = 'fits')
final_table.write(output_file + '.csv', format = 'csv')
merged_log.write(output_file + ' log.csv', format = 'csv')
print('Script finished')
